In [96]:
def analyse(path):
    import os
    import pandas as pd
  #  import numpy as np
    
    os.chdir(path)
    os.mkdir('output')
    new_path=os.path.join(path,'output')
    
    nic = pd.read_csv("ncbiid.csv",header=None)
    names = pd.read_csv("org.csv", sep=" ", header=None)

    
    for infile in os.listdir(path):
        if infile.endswith("txt"):
            s_name=infile.split('.')[0]
            f_name=s_name.split('_')[1]
            
            file = pd.read_csv(infile, sep="\t", header=None)
            file = file.assign(**{"ncbi_id": nic,"names":names})
            file.rename(columns = {0:'Organism',1:'Genome_size',2: f'{f_name}_mapped_read_segments', 3:'unmapped_read_segments'}, inplace=True)

            # keep rows where mapped reads is not zero
            new_file = file.loc[file[f'{f_name}_mapped_read_segments'] > 0, :]
       #     new_file = file[file['mapped_read_segments'] > 0]
            
            # add read length and genome coverage columns 
            new_file["read_length"] = [161 for i in range(len(new_file['Organism']))]
            
            # calculate genome coverage and then add to the data frame
            cal = (new_file[f'{f_name}_mapped_read_segments'] * new_file['read_length']) / new_file['Genome_size']
            new_file["genome_coverage"] = cal
            
            # select rows with > 10 genome coverage and save to a new file
            final_file = new_file.loc[new_file["genome_coverage"] > 10, :]
            #final_file = new_file[new_file["genome_coverage"] > 10]
            final_file.to_csv(f'{new_path}/{infile}.csv',index=False)
            

            
            
            
            

In [110]:
# analyse('/Users/priyalakra/Desktop/covid_data_results/')

In [111]:
# merge two or more dataframes by common keys! 🤓
def combinef(path):
    
    import os
    import pandas as pd
    from functools import reduce
    os.chdir(path)
    os.mkdir('output2')
    new_path=os.path.join(path,'output2')
    
    names = []
    for infile in os.listdir(path):
        if infile.endswith("csv"):
            s_name=infile.split('.')[0]
            f_name=s_name.split('_')[1]
            file = pd.read_csv(infile)
            new_file = file[['Organism',f'{f_name}_mapped_read_segments']]
            names.append(new_file)
    
    namesf = reduce(lambda  left,right: pd.merge(left,right,on=['Organism'],
                                            how='outer'), names)  # outer: use union of keys from both frames
    namesf.to_csv(f"{new_path}/file.csv", index=False)
    

In [112]:
#combinef('/Users/priyalakra/Desktop/covid_data_results/output')

In [113]:
# simply concatenate two or more dataframes by columns 
def combine(path):
    
    import os
    import pandas as pd
    os.chdir(path)
    os.mkdir('output2')
    new_path=os.path.join(path,'output2')
    
    names = []
    for infile in os.listdir(path):
        if infile.endswith("csv"):
            file = pd.read_csv(infile)
            new_file = file[['Organism','mapped_read_segments']]
            names.append(new_file)
    
    names = pd.concat(names, axis=1)
    names.to_csv(f"{new_path}/file.csv", index=False)


In [114]:
# combine('/Users/priyalakra/Desktop/covid_data_results/output')